In [1]:
import json 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

In [2]:
with open(r"F:\Python\NLP\First_aid_bot\archive (2)\intents.json",'r') as file:
    data=json.load(file)

In [3]:
df=pd.DataFrame(data['intents'])

df

,tag,patterns,responses,context_set
0,Cuts,"[What to do if Cuts?, How to cure Cuts?, Which...",[Wash the cut properly to prevent infection an...,
1,Abrasions,"[how do you treat abrasions?, Do Abrasions cau...",[Begin with washed hands.Gently clean the area...,
2,stings,"[How do you treat Sting?, Stings, What to do i...",[Remove any stingers immediately. Some experts...,
3,Splinter,"[How to remove Splinters, How to cure Splinter...",[1. SOAK IT IN EPSOM SALTS. Dissolve a cup of ...,
4,Sprains,"[How do you treat a sprain?, what to do if i g...",[Use an ice pack or ice slush bath immediately...,
5,Strains,"[How do you treat a strain?, what to do if i g...","[Rest,Ice,Compression and Elevation can be use...",
6,Fever,"[How do you treat a mild Fever?, what to do if...",[To treat a fever at home: 1)Drink plenty of f...,
7,Nasal Congestion,"[How do you treat nasal Congestion?, what to d...","[When you’re stuffed up, focus on keeping your...",
8,Cough,"[How to cure cough?, How do you treat cough?, ...","[1) Honey:- Use honey to treat a cough, mix 2 ...",
9,Sore Throat,"[How do you treat sore throat?, what to do if ...",[1) Make sure you get plenty of rest and drink...,


In [4]:
df.drop(df.index[18:36],inplace=True)

In [5]:
df.reset_index(drop=True,inplace=True)

In [6]:
df

,tag,patterns,responses,context_set
0,Cuts,"[What to do if Cuts?, How to cure Cuts?, Which...",[Wash the cut properly to prevent infection an...,
1,Abrasions,"[how do you treat abrasions?, Do Abrasions cau...",[Begin with washed hands.Gently clean the area...,
2,stings,"[How do you treat Sting?, Stings, What to do i...",[Remove any stingers immediately. Some experts...,
3,Splinter,"[How to remove Splinters, How to cure Splinter...",[1. SOAK IT IN EPSOM SALTS. Dissolve a cup of ...,
4,Sprains,"[How do you treat a sprain?, what to do if i g...",[Use an ice pack or ice slush bath immediately...,
5,Strains,"[How do you treat a strain?, what to do if i g...","[Rest,Ice,Compression and Elevation can be use...",
6,Fever,"[How do you treat a mild Fever?, what to do if...",[To treat a fever at home: 1)Drink plenty of f...,
7,Nasal Congestion,"[How do you treat nasal Congestion?, what to d...","[When you’re stuffed up, focus on keeping your...",
8,Cough,"[How to cure cough?, How do you treat cough?, ...","[1) Honey:- Use honey to treat a cough, mix 2 ...",
9,Sore Throat,"[How do you treat sore throat?, what to do if ...",[1) Make sure you get plenty of rest and drink...,


In [7]:
df['patterns'][16]

['How do you treat a wound?',
 'what to do if i get a Wound?',
 'Which medicine to take if I get wounded?',
 'How to cure a wound?']

Chatbot

In [8]:

def chatbot(a):
    for i,row in df.iterrows():
        found_match=False
        for pattern_list in row['patterns']:
            for word in a.split(" "):
                if word in pattern_list:
                    return df['responses'][i][0]
                    found_match=True
                    break
            if found_match:
                break


def user_input():
    return input("What Happened tell me I am here")

while True: 
    a=user_input()
    
    if a.lower() in ["exit","quit"]:
        print("see you soon buddy")
        break
    else:
        response=chatbot(a)
        print(response)
       
        
                

Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
see you soon buddy
